Известно, что FNN сетка вполне хорошо настраивается на распознование рукописных цифр (датасет mnist). На одном из уроков мы смогли подобрать параметры для 97.7%-ой точности предсказаний уже на 5 эпохах обучения:

Epoch 5/5
60000/60000 [==============================] - 19s 311us/step - loss: 0.0374 - accuracy: 0.9884, test: [0.073, 0.9775]

В литературе говорится о значительном положительном влиянии dropout на достижение еще большей точности на тестовом наборе.
Но этот прием применяется для временного отключения случайного набора внутренних нейронов. А что произойдет с точностью обучения, когда мы по такому же случайному принципу будем зашумливать входящий сигнал? Своего рода kick-out элементов входного сигнала... Это можно считать проверкой на помехоустойчивость сети. Будем использовать те же 5 эпох, и случайным образом инвертировать сигнал в n процентах входящих пикселях тренировочных образцов. 

In [ ]:
# The full neural network code!
###############################
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

(train_images,train_labels),(test_images,test_labels) = mnist.load_data()

# Normalize the images.
train_images = (train_images / 255) 
test_images  = ( test_images / 255) 

# Flatten the images.
train_images_base = train_images.reshape((-1, 784))
test_images  =  test_images.reshape((-1, 784))

In [16]:
KI,KP = train_images.shape
print(f'KImages = {KI}, KPixels = {KP}')

KImages = 60000, KPixels = 784


In [38]:
n = 0.50       #0.01 - 0.5
K = int(KP*n)
i_base = [en for en in range(KP)]

train_ims = train_images.copy()
i = i_base.copy()

In [39]:
for j in range(KI):
    np.random.shuffle(i)
    for ij in range(K):
        train_ims[j,i[ij]] = 1.0 - train_ims[j,i[ij]] 
print(f'K= {K}')
lst = i[0:K]
print(lst)

K= 392
[284, 335, 151, 663, 277, 215, 514, 43, 703, 379, 88, 675, 391, 87, 361, 587, 82, 439, 236, 513, 316, 344, 737, 254, 646, 133, 350, 677, 179, 275, 741, 279, 528, 424, 211, 740, 607, 408, 289, 161, 164, 533, 174, 320, 259, 441, 256, 730, 206, 94, 71, 217, 736, 147, 18, 773, 769, 80, 522, 52, 595, 300, 755, 635, 390, 64, 101, 317, 777, 700, 239, 180, 32, 759, 704, 525, 282, 241, 628, 711, 381, 294, 193, 728, 69, 207, 205, 485, 308, 545, 599, 65, 202, 185, 502, 673, 637, 725, 160, 276, 413, 446, 456, 770, 523, 772, 269, 591, 564, 612, 463, 549, 588, 250, 655, 315, 472, 233, 227, 688, 342, 417, 761, 760, 57, 505, 1, 338, 689, 302, 654, 114, 412, 575, 382, 267, 337, 393, 395, 622, 638, 626, 260, 397, 576, 461, 619, 76, 380, 121, 743, 487, 340, 204, 722, 754, 314, 479, 98, 465, 448, 531, 444, 610, 212, 359, 471, 288, 135, 524, 672, 735, 197, 684, 334, 434, 331, 220, 377, 311, 209, 432, 213, 93, 697, 409, 226, 223, 142, 403, 692, 173, 573, 102, 606, 321, 483, 768, 714, 661, 312, 431, 1

In [40]:
# Build the model.
model = Sequential([
  Dense(512, activation='relu', input_shape=(784,)), # 64
  Dense(128, activation='relu'),                     # 64
  Dense( 32, activation='relu'),                     # 64
  Dense( 10, activation='softmax'),                  # 10
]) 

# Compile the model.
model.compile(  optimizer='adam',  loss='categorical_crossentropy',  metrics=['accuracy'])

# Train the model.
model.fit(  train_ims,  to_categorical(train_labels),  epochs=5,  batch_size=32)

# Evaluate the model.
model.evaluate(  test_images,  to_categorical(test_labels))

Epoch 1/5
60000/60000 [==============================] - 19s 325us/step - loss: 2.3019 - accuracy: 0.1115
Epoch 2/5
60000/60000 [==============================] - 20s 326us/step - loss: 2.3020 - accuracy: 0.1122
Epoch 3/5
60000/60000 [==============================] - 22s 365us/step - loss: 2.3014 - accuracy: 0.1124
Epoch 4/5
60000/60000 [==============================] - 23s 389us/step - loss: 2.3014 - accuracy: 0.1124
Epoch 5/5
10000/10000 [==============================] - 1s 74us/step


[2.301010782623291, 0.11349999904632568]

In [ ]:
  0%  train loss: 0.0374 - accuracy: 0.9884, test: [0.073, 0.9775]
  1%  train loss: 0.0271 - accuracy: 0.9917, test: [0.099, 0.9754]
  5%  train loss: 0.0281 - accuracy: 0.9904, test: [0.124, 0.9747]
 10%  train loss: 0.0359 - accuracy: 0.9882, test: [0.179, 0.9689]
 15%  train loss: 0.0544 - accuracy: 0.9816, test: [0.257, 0.9625]
 20%  train loss: 0.0899 - accuracy: 0.9693, test: [0.340, 0.9538]
 30%  train loss: 0.4240 - accuracy: 0.8565, test: [0.476, 0.9262]
 40%  train loss: 2.3013 - accuracy: 0.1124, test: [2.256, 0.1136]
 50%  train loss: 2.3014 - accuracy: 0.1124, test: [2.301, 0.1134]


In [ ]:
'''
Выводы:
Случайный шум оставляет возможность хорошего обучения модели FNN вплоть до 30% замены интенсивности пикселов на "обратную".
Если надо быстрее зашумить изображения - лучше проводить протяженные (неслучайные) куски линий.
При повышении процента шума модель первым делом попадает в режим переобучения, после чего (с процентом шума 20)
- переходит в стадию недообучения, а потом предсказательная способность окончательно рушится
'''
    